In [1]:
import pandas as pd
import MalekFinance as mf
import datetime as dt
from pandas.tseries.offsets import MonthEnd
from tqdm.notebook import tqdm
import numpy as np
import statsmodels.api as sm
from matplotlib import pyplot as plt
from sklearn.decomposition import PCA
pd.set_option('display.float_format', '{:.4f}'.format)

In [2]:
monthly_returns = mf.read('WRDS Data','All Exchanges CRSP PEMRMNO Monthly Returns DataFrame 63-22',index_col=0,parse_dates=True)
Returns_Check_NYSE_df = mf.read('WRDS Data','NYSE Monthly Returns',index_col=0,parse_dates=True)
monthly_size = mf.read('WRDS Data','All Exchanges CRSP PEMRMNO Monthly Size DataFrame 63-22',index_col=0,parse_dates=True)
NYSE_Size_df = monthly_size[monthly_size.columns.intersection(Returns_Check_NYSE_df.columns)]
Two_Year_Data_Check = monthly_returns.rolling(24,min_periods=24).std()*(12**0.5)
NYSE_Two_Year_Data_Check = Two_Year_Data_Check[Two_Year_Data_Check.columns.intersection(Returns_Check_NYSE_df.columns)]
past_11 = mf.past_x(monthly_returns,11)
NYSE_Past_11_df = mf.past_x(Returns_Check_NYSE_df,11)
formation_dates = pd.date_range('1970-12-31','2022-11-30',freq='M')
returns_dates = pd.date_range('1971-1-31','2022-12-31',freq='M')
formation_dates1 = pd.date_range(start='1970-12-31', end='2022-11-30', freq='2Q')
past_48 = mf.past_x(monthly_returns,48)
NYSE_past_48 = mf.past_x(Returns_Check_NYSE_df,48)
def compute_weights(df_):
    return df_['Market Cap'] / df_['Market Cap'].sum()

# Long Term Reversal

In [3]:
def LTRF(formation, holding):
    NYSE_Size = NYSE_Size_df.loc[formation].dropna()
    NYSE_Past_48 = NYSE_past_48.loc[formation - MonthEnd(12)].dropna()
    Returns_Check_NYSE = Returns_Check_NYSE_df.loc[formation + MonthEnd(1)].dropna()
    NYSE_Merged = pd.concat([Returns_Check_NYSE,NYSE_Past_48,NYSE_Size],axis=1).dropna()
    NYSE_Merged.columns = ['1 Month Forward Return','Past 48','Market Cap']
    Long_Threshold = NYSE_Merged['Past 48'].quantile(0.3)
    Short_Threshold = NYSE_Merged['Past 48'].quantile(0.7)
    Size_Breakpoint = NYSE_Merged['Market Cap'].median()
    All_Size = monthly_size.loc[formation ].dropna()
    All_Past_48 = past_48.loc[formation - MonthEnd(12)].dropna()
    All_Returns = monthly_returns.loc[formation + MonthEnd(1)].dropna()
    All_Merged = pd.concat([All_Returns,All_Past_48,All_Size],axis=1).dropna()
    All_Merged.columns = ['1 Month Forward Return','Past 48','Market Cap']
    Large = All_Merged[All_Merged['Market Cap'] >= Size_Breakpoint]
    Small = All_Merged[All_Merged['Market Cap'] <= Size_Breakpoint]
    Large_Long = Large[Large['Past 48'] <= Long_Threshold]
    Large_Short = Large[Large['Past 48'] >= Short_Threshold]
    Small_Long = Small[Small['Past 48'] <= Long_Threshold]
    Small_Short = Small[Small['Past 48'] >= Short_Threshold]
    Large_Long_weights  = compute_weights(Large_Long)
    Large_Short_weights = compute_weights(Large_Short)
    Small_Long_weights  = compute_weights(Small_Long)
    Small_Short_weights = compute_weights(Small_Short)
    Portfolio_Return = []
    for m in range(1, holding + 1):
        date_m = formation + MonthEnd(m)
        large_long_ret  = monthly_returns.loc[date_m, Large_Long.index].mul(Large_Long_weights, fill_value=0).sum()
        large_short_ret = monthly_returns.loc[date_m, Large_Short.index].mul(Large_Short_weights, fill_value=0).sum()
        small_long_ret  = monthly_returns.loc[date_m, Small_Long.index].mul(Small_Long_weights, fill_value=0).sum()
        small_short_ret = monthly_returns.loc[date_m, Small_Short.index].mul(Small_Short_weights, fill_value=0).sum()
        portfolio_ret = ((large_long_ret + small_long_ret) - (large_short_ret + small_short_ret)) / 2.0
        Portfolio_Return.append(portfolio_ret)
    return Portfolio_Return

LTR = pd.DataFrame(data={'Long-Term Reversal': np.concatenate([LTRF(i,6) for i in tqdm(formation_dates1)])}, index=returns_dates)

  0%|          | 0/104 [00:00<?, ?it/s]

In [4]:
def LT_Residual_Reversal_PCA(formation, holding):
    NYSE_Size = NYSE_Size_df.loc[formation].dropna()
    Returns_Check_NYSE = Returns_Check_NYSE_df.loc[formation + MonthEnd(1)].dropna()
    NYSE_Past_48 = NYSE_past_48.loc[formation - MonthEnd(12)].dropna()
    NYSE_Merged = pd.concat([Returns_Check_NYSE, NYSE_Size, NYSE_Past_48], axis=1).dropna()
    NYSE_Merged.columns = ['1 Month Forward Return','Market Cap','Past 48']
    Size_Breakpoint = NYSE_Merged['Market Cap'].median()
    All_Size = monthly_size.loc[formation].dropna()
    All_Returns = monthly_returns.loc[formation + MonthEnd(1)].dropna()
    All_Past_48 = past_48.loc[formation - MonthEnd(12)].dropna()
    All_Merged = pd.concat([All_Returns, All_Size, All_Past_48], axis=1).dropna()
    All_Merged.columns = ['1 Month Forward Return','Market Cap','Past 48'] 
    df = monthly_returns.loc[formation - MonthEnd(59) : formation,  monthly_returns.columns.intersection(All_Merged.index)].fillna(0)
    pca_transformed = PCA(n_components=5, random_state=0).fit_transform(df)
    APM = mf.Equal_Vol(pd.DataFrame(pca_transformed, index=df.index), 0.15)
    x = sm.add_constant(APM)
    resids = mf.residuals(df, x)
    residual_slice = resids.loc[formation - MonthEnd(59) : formation - MonthEnd(12)]
    Past_LTR_Residual = mf.past_x(residual_slice, 48).loc[formation - MonthEnd(12)]
    All_Merged = All_Merged.merge(Past_LTR_Residual, left_index=True, right_index=True, how='inner')
    All_Merged.rename(columns={All_Merged.columns[3]: 'Residual Past 48 Month'}, inplace=True)
    NYSE_Merged = NYSE_Merged.merge(Past_LTR_Residual, left_index=True, right_index=True, how='inner').dropna()
    NYSE_Merged.rename(columns={NYSE_Merged.columns[3]: 'Residual Past 48 Month'}, inplace=True)
    Long_Threshold = NYSE_Merged['Residual Past 48 Month'].quantile(0.3)
    Short_Threshold = NYSE_Merged['Residual Past 48 Month'].quantile(0.7)
    Large = All_Merged[All_Merged['Market Cap'] >= Size_Breakpoint]
    Small = All_Merged[All_Merged['Market Cap'] <=  Size_Breakpoint]
    Large_Long  = Large[Large['Residual Past 48 Month'] <= Long_Threshold]
    Large_Short = Large[Large['Residual Past 48 Month'] >= Short_Threshold]
    Small_Long  = Small[Small['Residual Past 48 Month'] <= Long_Threshold]
    Small_Short = Small[Small['Residual Past 48 Month'] >= Short_Threshold]
    Large_Long_weights  = compute_weights(Large_Long)
    Large_Short_weights = compute_weights(Large_Short)
    Small_Long_weights  = compute_weights(Small_Long)
    Small_Short_weights = compute_weights(Small_Short)
    Portfolio_Return = []
    for m in range(1, holding + 1):
        date_m = formation + MonthEnd(m)
        large_long_ret  = monthly_returns.loc[date_m, Large_Long.index].mul(Large_Long_weights, fill_value=0).sum()
        large_short_ret = monthly_returns.loc[date_m, Large_Short.index].mul(Large_Short_weights, fill_value=0).sum()
        small_long_ret  = monthly_returns.loc[date_m, Small_Long.index].mul(Small_Long_weights, fill_value=0).sum()
        small_short_ret = monthly_returns.loc[date_m, Small_Short.index].mul(Small_Short_weights, fill_value=0).sum()
        portfolio_ret = ((large_long_ret + small_long_ret) - (large_short_ret + small_short_ret)) / 2.0
        Portfolio_Return.append(portfolio_ret)
    return Portfolio_Return

Res_LTREV_PCA = pd.DataFrame(data={'Residual Long-Term Reversal PCA': np.concatenate([LT_Residual_Reversal_PCA(i,6) for i in tqdm(formation_dates1)])}, index=returns_dates)

  0%|          | 0/104 [00:00<?, ?it/s]

# Short Term Reversal

In [5]:
def STRF(formation):
    NYSE_Size = NYSE_Size_df.loc[formation].dropna()
    NYSE_Past_1 = Returns_Check_NYSE_df.loc[formation].dropna()
    Returns_Check_NYSE = Returns_Check_NYSE_df.loc[formation + MonthEnd(1)].dropna()
    NYSE_Merged = pd.concat([Returns_Check_NYSE,NYSE_Past_1,NYSE_Size],axis=1).dropna()
    NYSE_Merged.columns = ['1 Month Forward Return','Past 1','Market Cap']
    Winners_Threshold = NYSE_Merged['Past 1'].quantile(0.3)
    Losers_Threshold = NYSE_Merged['Past 1'].quantile(0.7)
    Size_Breakpoint = NYSE_Merged['Market Cap'].median()
    All_Size = monthly_size.loc[formation].dropna()
    All_Past_1 = monthly_returns.loc[formation].dropna()
    All_Returns = monthly_returns.loc[formation + MonthEnd(1)].dropna()
    All_Merged = pd.concat([All_Returns,All_Past_1,All_Size],axis=1).dropna()
    All_Merged.columns = ['1 Month Forward Return','Past 1','Market Cap']
    Large = All_Merged[All_Merged['Market Cap'] >= Size_Breakpoint]
    Small = All_Merged[All_Merged['Market Cap'] <= Size_Breakpoint]
    Large_Winners = Large[Large['Past 1'] <= Winners_Threshold]
    Large_Losers = Large[Large['Past 1'] >= Losers_Threshold]
    Small_Winners = Small[Small['Past 1'] <= Winners_Threshold]
    Small_Losers = Small[Small['Past 1'] >= Losers_Threshold]
    Large_Winners_RET = sum(Large_Winners.iloc[:,0] * (Large_Winners.iloc[:,2]/sum(Large_Winners.iloc[:,2])))
    Large_Losers_RET = sum(Large_Losers.iloc[:,0] * (Large_Losers.iloc[:,2]/sum(Large_Losers.iloc[:,2])))
    Small_Winners_RET = sum(Small_Winners.iloc[:,0] * (Small_Winners.iloc[:,2]/sum(Small_Winners.iloc[:,2])))
    Small_Losers_RET = sum(Small_Losers.iloc[:,0] * (Small_Losers.iloc[:,2]/sum(Small_Losers.iloc[:,2])))
    Portfolio_Return = ((Large_Winners_RET + Small_Winners_RET) - (Large_Losers_RET + Small_Losers_RET))/2
    return Portfolio_Return

STR = pd.DataFrame(data={'Short-Term Reversal':[STRF(i) for i in tqdm(formation_dates)]},index=returns_dates)

  0%|          | 0/624 [00:00<?, ?it/s]

In [6]:
def ST_Residual_Reversal_PCA(formation):
    NYSE_Size = NYSE_Size_df.loc[formation].dropna()
    Returns_Check_NYSE = Returns_Check_NYSE_df.loc[formation + MonthEnd(1)].dropna()
    NYSE_Two_Year = NYSE_Two_Year_Data_Check.loc[formation].dropna()
    NYSE_Merged = pd.concat([Returns_Check_NYSE,NYSE_Size,NYSE_Two_Year],axis=1).dropna()
    NYSE_Merged.columns = ['1 Month Forward Return','Market Cap','Two Year Check']
    Size_Breakpoint = NYSE_Merged['Market Cap'].median()
    All_Size = monthly_size.loc[formation].dropna()
    All_Returns = monthly_returns.loc[formation + MonthEnd(1)].dropna()
    All_Two_Year = Two_Year_Data_Check.loc[formation].dropna()
    All_Merged = pd.concat([All_Returns,All_Size,All_Two_Year],axis=1).dropna()
    All_Merged.columns = ['1 Month Forward Return','Market Cap','Two Year Check']
    df = monthly_returns[monthly_returns.columns.intersection(All_Merged.index)][formation - MonthEnd(23):formation].fillna(0)
    APM = mf.Equal_Vol(pd.DataFrame(PCA(5).fit_transform(df)),0.15)
    x = sm.add_constant(APM)
    resids = mf.residuals(df,x)
    Past_Residual_1 = resids.loc[formation]
    NYSE_Merged = NYSE_Merged.merge(Past_Residual_1,left_index=True,right_index=True,how='inner').dropna()
    NYSE_Merged.rename(columns={NYSE_Merged.columns[3]:'Residual Past 1 Month'},inplace=True)
    Long_Threshold = NYSE_Merged['Residual Past 1 Month'].quantile(0.3)
    Short_Threshold = NYSE_Merged['Residual Past 1 Month'].quantile(0.7)
    All_Merged = All_Merged.merge(Past_Residual_1,left_index=True,right_index=True,how='inner').dropna()
    All_Merged.rename(columns={All_Merged.columns[3]:'Residual Past 1 Month'},inplace=True)
    Large = All_Merged[All_Merged['Market Cap'] >= Size_Breakpoint]
    Small = All_Merged[All_Merged['Market Cap'] <= Size_Breakpoint]
    Large_Long = Large[Large['Residual Past 1 Month'] <= Long_Threshold]
    Large_Short = Large[Large['Residual Past 1 Month'] >= Short_Threshold]
    Small_Long = Small[Small['Residual Past 1 Month'] <= Long_Threshold]
    Small_Short = Small[Small['Residual Past 1 Month'] >= Short_Threshold]
    for i in Large_Long,Small_Long,Large_Short,Small_Short:
        i['Weight'] = i['Market Cap']/sum(i['Market Cap'])
    Large_Long_RET = sum(Large_Long['1 Month Forward Return'] * Large_Long['Weight'])
    Large_Short_RET = sum(Large_Short['1 Month Forward Return'] * Large_Short['Weight'])
    Small_Long_RET = sum(Small_Long['1 Month Forward Return'] * Small_Long['Weight'])
    Small_Short_RET = sum(Small_Short['1 Month Forward Return'] * Small_Short['Weight'])
    Portfolio_Return = ((Large_Long_RET + Small_Long_RET) - (Large_Short_RET + Small_Short_RET))/2
    return Portfolio_Return

Res_STREV_PCA = pd.DataFrame(data={'Residual Short-Term Reversal PCA':[ST_Residual_Reversal_PCA(i) for i in tqdm(formation_dates)]},index=returns_dates)

  0%|          | 0/624 [00:00<?, ?it/s]

# Momentum

In [7]:
def WMLF(formation):
    NYSE_Size = NYSE_Size_df.loc[formation].dropna()
    NYSE_Past_11 = NYSE_Past_11_df.loc[formation - MonthEnd(1)].dropna()
    Returns_Check_NYSE = Returns_Check_NYSE_df.loc[formation + MonthEnd(1)].dropna()
    NYSE_Merged = pd.concat([Returns_Check_NYSE,NYSE_Past_11,NYSE_Size],axis=1).dropna()
    NYSE_Merged.columns = ['2 Month Forward Return','Past 11','Market Cap']
    Winners_Threshold = NYSE_Merged['Past 11'].quantile(0.7)
    Losers_Threshold = NYSE_Merged['Past 11'].quantile(0.3)
    Size_Breakpoint = NYSE_Merged['Market Cap'].median()
    All_Size = monthly_size.loc[formation].dropna()
    All_Past_11 = past_11.loc[formation - MonthEnd(1)].dropna()
    All_Returns = monthly_returns.loc[formation + MonthEnd(1)].dropna()
    All_Merged = pd.concat([All_Returns,All_Past_11,All_Size],axis=1).dropna()
    All_Merged.columns = ['2 Month Forward Return','Past 11','Market Cap']
    Large = All_Merged[All_Merged['Market Cap'] >= Size_Breakpoint]
    Small = All_Merged[All_Merged['Market Cap'] <= Size_Breakpoint]
    Large_Winners = Large[Large['Past 11'] >= Winners_Threshold]
    Large_Losers = Large[Large['Past 11'] <= Losers_Threshold]
    Small_Winners = Small[Small['Past 11'] >= Winners_Threshold]
    Small_Losers = Small[Small['Past 11'] <= Losers_Threshold]
    Large_Winners_RET = sum(Large_Winners.iloc[:,0] * (Large_Winners.iloc[:,2]/sum(Large_Winners.iloc[:,2])))
    Large_Losers_RET = sum(Large_Losers.iloc[:,0] * (Large_Losers.iloc[:,2]/sum(Large_Losers.iloc[:,2])))
    Small_Winners_RET = sum(Small_Winners.iloc[:,0] * (Small_Winners.iloc[:,2]/sum(Small_Winners.iloc[:,2])))
    Small_Losers_RET = sum(Small_Losers.iloc[:,0] * (Small_Losers.iloc[:,2]/sum(Small_Losers.iloc[:,2])))
    Portfolio_Return = ((Large_Winners_RET + Small_Winners_RET) - (Large_Losers_RET + Small_Losers_RET))/2
    return Portfolio_Return

WML = pd.DataFrame(data={'Momentum':[WMLF(i) for i in tqdm(formation_dates)]},index=returns_dates)

  0%|          | 0/624 [00:00<?, ?it/s]

In [8]:
def Residual_Momentum_PCA(formation):
    NYSE_Size = NYSE_Size_df.loc[formation].dropna()
    Returns_Check_NYSE = Returns_Check_NYSE_df.loc[formation + MonthEnd(1)].dropna()
    NYSE_Two_Year = NYSE_Two_Year_Data_Check.loc[formation].dropna()
    NYSE_Merged = pd.concat([Returns_Check_NYSE,NYSE_Size,NYSE_Two_Year],axis=1).dropna()
    NYSE_Merged.columns = ['1 Month Forward Return','Market Cap','Two Year Check']
    Size_Breakpoint = NYSE_Merged['Market Cap'].median()
    All_Size = monthly_size.loc[formation].dropna()
    All_Returns = monthly_returns.loc[formation + MonthEnd(1)].dropna()
    All_Two_Year = Two_Year_Data_Check.loc[formation].dropna()
    All_Merged = pd.concat([All_Returns,All_Size,All_Two_Year],axis=1).dropna()
    All_Merged.columns = ['1 Month Forward Return','Market Cap','Two Year Check']
    df = monthly_returns[monthly_returns.columns.intersection(All_Merged.index)][formation - MonthEnd(23):formation].fillna(0)
    APM = mf.Equal_Vol(pd.DataFrame(PCA(5).fit_transform(df)),0.15)
    x = sm.add_constant(APM)
    resids = mf.residuals(df,x)
    Past_Residual_11 = mf.past_x(resids[-12:],11).loc[formation - MonthEnd(1)]
    NYSE_Merged = NYSE_Merged.merge(Past_Residual_11,left_index=True,right_index=True,how='inner').dropna()
    NYSE_Merged.rename(columns={NYSE_Merged.columns[3]:'Residual Past 11 Month'},inplace=True)
    Long_Threshold = NYSE_Merged['Residual Past 11 Month'].quantile(0.7)
    Short_Threshold = NYSE_Merged['Residual Past 11 Month'].quantile(0.3)
    All_Merged = All_Merged.merge(Past_Residual_11,left_index=True,right_index=True,how='inner').dropna()
    All_Merged.rename(columns={All_Merged.columns[3]:'Residual Past 11 Month'},inplace=True)
    Large = All_Merged[All_Merged['Market Cap'] >= Size_Breakpoint]
    Small = All_Merged[All_Merged['Market Cap'] <= Size_Breakpoint]
    Large_Long = Large[Large['Residual Past 11 Month'] >= Long_Threshold]
    Large_Short = Large[Large['Residual Past 11 Month'] <= Short_Threshold]
    Small_Long = Small[Small['Residual Past 11 Month'] >= Long_Threshold]
    Small_Short = Small[Small['Residual Past 11 Month'] <= Short_Threshold]
    for i in Large_Long,Small_Long,Large_Short,Small_Short:
        i['Weight'] = i['Market Cap']/sum(i['Market Cap'])
    Large_Long_RET = sum(Large_Long['1 Month Forward Return'] * Large_Long['Weight'])
    Large_Short_RET = sum(Large_Short['1 Month Forward Return'] * Large_Short['Weight'])
    Small_Long_RET = sum(Small_Long['1 Month Forward Return'] * Small_Long['Weight'])
    Small_Short_RET = sum(Small_Short['1 Month Forward Return'] * Small_Short['Weight'])
    Portfolio_Return = ((Large_Long_RET + Small_Long_RET) - (Large_Short_RET + Small_Short_RET))/2
    return Portfolio_Return

Res_MOM_PCA = pd.DataFrame(data={'Residual Momentum PCA':[Residual_Momentum_PCA(i) for i in tqdm(formation_dates)]},index=returns_dates)

  0%|          | 0/624 [00:00<?, ?it/s]

In [9]:
mf.summary_df(pd.concat([LTR,Res_LTREV_PCA],axis=1).dropna())

,Long-Term Reversal,Residual Long-Term Reversal PCA
Annual Return,2.1800,4.4500
Annual Volity,9.0100,8.7700
Sharpe Ratio,0.2400,0.5100
Max Drawdown,-51.1100,-28.1000
Portfolio Beta,0.0100,0.1600


In [10]:
mf.summary_df(pd.concat([STR,Res_STREV_PCA],axis=1).dropna())

,Short-Term Reversal,Residual Short-Term Reversal PCA
Annual Return,5.0800,8.0200
Annual Volity,11.1600,6.7300
Sharpe Ratio,0.4600,1.1900
Max Drawdown,-32.9100,-13.1800
Portfolio Beta,0.2300,0.1000


In [11]:
mf.summary_df(pd.concat([WML,Res_MOM_PCA],axis=1).dropna())

,Momentum,Residual Momentum PCA
Annual Return,9.1500,8.4700
Annual Volity,14.3600,6.9400
Sharpe Ratio,0.6400,1.2200
Max Drawdown,-53.3700,-22.3800
Portfolio Beta,-0.1600,-0.1000
